
# Baseline performance with three models:
- IndicBART
- mT5 multilingual_XLSum
- mT5 base Google

#### **Installing dependencies**

In [ ]:
!pip install transformers sentencepiece datasets rouge -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 43.3 MB/s eta 0:00:00


#### **Importing libraries**

In [ ]:
import torch
import pandas as pd
from rouge import Rouge
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import plotly.express as px
import matplotlib.pyplot as plt

from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

#### **Downloading the data**

In [ ]:
import os

from google.colab import drive
drive.mount('/content/drive')
# Set your working directory to a folder in your Google Drive. This way, if your notebook times out,
# your files will be saved in your Google Drive!

## If you are using the data by mounting the google drive, use the following :
from google.colab import drive
drive.mount('/content/gdrive')

# the base Google Drive directory
root_dir = "/content/gdrive/MyDrive/"

# choose where you want your project files to be saved
project_folder = "Colab Notebooks/NLP_Project/"

def create_and_set_working_directory(project_folder):
  # check if your project folder exists. if not, it will be created.
  if os.path.isdir(root_dir + project_folder) == False:
    os.mkdir(root_dir + project_folder)
    print(root_dir + project_folder + ' did not exist but was created.')

  # change the OS to use your project folder as the working directory
  os.chdir(root_dir + project_folder)

  # create a test file to make sure it shows up in the right place
  !touch 'new_file_in_working_directory.txt'
  print('\nYour working directory was changed to ' + root_dir + project_folder + \
        "\n\nAn empty text file was created there. You can also run !pwd to confirm the current working directory." )

create_and_set_working_directory(project_folder)
path = root_dir + project_folder

Mounted at /content/drive
Mounted at /content/gdrive

Your working directory was changed to /content/gdrive/MyDrive/Colab Notebooks/NLP_Project/

An empty text file was created there. You can also run !pwd to confirm the current working directory.


In [ ]:
data = pd.read_csv(path + "datasets/hindi_train.csv")

In [ ]:
data

,Id,Heading,Summary,Article
0,hindi_2023_train_0,"गला दबाकर हत्या की; बॉडी बोरे में भरी, लोकल मा...",Kerala Minor Girl Rape Case - केरल के एर्नाकुल...,केरल के एर्नाकुलम जिले में 5 साल की बच्ची से र...
1,hindi_2023_train_1,तेलंगाना में 18 की जान गई; जम्मू-कश्मीर में बा...,इस साल मानसून सीजन में कई राज्यों में भारी तबा...,मानसून सीजन में हुई भारी बारिश ने कई राज्यों म...
2,hindi_2023_train_2,"राजस्थान सरकार बनाएगी कर्ज राहत आयोग, कोर्ट के...",चुनावी साल में राजस्थान सरकार किसानों को लुभान...,चुनावी साल में राजस्थान सरकार किसानों को लुभान...
3,hindi_2023_train_3,"3 से 7 अगस्त तक कर सकेंगे अप्लाय, प्राइस बैंड ...",Non-banking lender SBFC Finance's initial publ...,नॉन बैंकिग फाइनेंस कंपनी 'SBFC फाइनेंस लिमिटेड...
4,hindi_2023_train_4,"डाइनिंग टेबल पर कुकर-कड़ाही न सजाएं, चीन के खा...",स्वाद खाने की बुनियाद है। लेकिन अगर खाना सुंदर...,स्वाद खाने की बुनियाद है। लेकिन अगर खाना सुंदर...
...,...,...,...,...
21220,hindi_2022_11363,"MSP पर प्रधानमंत्री ने कमेटी बनाई, पराली जलाना...","कृषि मंत्री ने कहा, तीनों कृषि कानूनों को वापस...",नई दिल्ली: कृषि मंत्री नरेंद्र सिंह तोमर ने एम...
21221,hindi_2022_11364,"योगी सरकार ने 9 गेस्ट हाउस का नाम नदियों, धार्...",मुंबई में स्टेट गेस्ट हाउस अब यूपी स्टेट गेस्ट...,लखनऊ: उत्तर प्रदेश की योगी आदित्यनाथ सरकार ने ...
21222,hindi_2022_11365,"Coronavirus से कुछ राहत, 7 लाख के नीचे आए एक्ट...",केंद्रीय स्वास्थ्य मंत्रालय की तरफ से शुक्रवार...,नई दिल्ली। कोरोना वायरस को लेकर कुछ राहत की बा...
21223,hindi_2022_11366,"वाराणसी गंगा नदी के किनारे सबसे साफ शहर बना, प...",लोकसभा में इस शहर का प्रतिनिधित्व करने वाले प्...,नई दिल्ली: केंद्रीय शहरी विकास मंत्री हरदीप सि...


In [ ]:
# Remove rows with empty or whitespace-only entries in critical columns
data = data.dropna(subset=['Heading', 'Summary', 'Article'])  # Remove rows where any of these columns are NaN
data = data[data['Heading'].str.strip().astype(bool) & data['Summary'].str.strip().astype(bool) & data['Article'].str.strip().astype(bool)]

# Alternatively, to handle multiple text columns at once where any text field being empty should result in a row being dropped
text_columns = ['Heading', 'Summary', 'Article']
data = data.dropna(subset=text_columns)  # Drop rows where any specified columns are NaN
data = data[data[text_columns].apply(lambda x: x.str.strip().astype(bool)).all(axis=1)]

# Display the cleaned dataframe
data


,Id,Heading,Summary,Article
0,hindi_2023_train_0,"गला दबाकर हत्या की; बॉडी बोरे में भरी, लोकल मा...",Kerala Minor Girl Rape Case - केरल के एर्नाकुल...,केरल के एर्नाकुलम जिले में 5 साल की बच्ची से र...
1,hindi_2023_train_1,तेलंगाना में 18 की जान गई; जम्मू-कश्मीर में बा...,इस साल मानसून सीजन में कई राज्यों में भारी तबा...,मानसून सीजन में हुई भारी बारिश ने कई राज्यों म...
2,hindi_2023_train_2,"राजस्थान सरकार बनाएगी कर्ज राहत आयोग, कोर्ट के...",चुनावी साल में राजस्थान सरकार किसानों को लुभान...,चुनावी साल में राजस्थान सरकार किसानों को लुभान...
3,hindi_2023_train_3,"3 से 7 अगस्त तक कर सकेंगे अप्लाय, प्राइस बैंड ...",Non-banking lender SBFC Finance's initial publ...,नॉन बैंकिग फाइनेंस कंपनी 'SBFC फाइनेंस लिमिटेड...
4,hindi_2023_train_4,"डाइनिंग टेबल पर कुकर-कड़ाही न सजाएं, चीन के खा...",स्वाद खाने की बुनियाद है। लेकिन अगर खाना सुंदर...,स्वाद खाने की बुनियाद है। लेकिन अगर खाना सुंदर...
...,...,...,...,...
21220,hindi_2022_11363,"MSP पर प्रधानमंत्री ने कमेटी बनाई, पराली जलाना...","कृषि मंत्री ने कहा, तीनों कृषि कानूनों को वापस...",नई दिल्ली: कृषि मंत्री नरेंद्र सिंह तोमर ने एम...
21221,hindi_2022_11364,"योगी सरकार ने 9 गेस्ट हाउस का नाम नदियों, धार्...",मुंबई में स्टेट गेस्ट हाउस अब यूपी स्टेट गेस्ट...,लखनऊ: उत्तर प्रदेश की योगी आदित्यनाथ सरकार ने ...
21222,hindi_2022_11365,"Coronavirus से कुछ राहत, 7 लाख के नीचे आए एक्ट...",केंद्रीय स्वास्थ्य मंत्रालय की तरफ से शुक्रवार...,नई दिल्ली। कोरोना वायरस को लेकर कुछ राहत की बा...
21223,hindi_2022_11366,"वाराणसी गंगा नदी के किनारे सबसे साफ शहर बना, प...",लोकसभा में इस शहर का प्रतिनिधित्व करने वाले प्...,नई दिल्ली: केंद्रीय शहरी विकास मंत्री हरदीप सि...


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_data, val_data = train_test_split(data, test_size=0.1, random_state=42)

print("Training Set:")
print(train_data.head())
print("\nValidation Set:")
print(val_data.head())


Training Set:
                          Id  \
9479   hindi_2023_train_9479   
13586        hindi_2022_3729   
19938       hindi_2022_10081   
2879   hindi_2023_train_2879   
4325   hindi_2023_train_4325   

                                                 Heading  \
9479   कमजोर सीटों पर आएंगे मोदी-शाह, जानें कहां नया ...   
13586  अंबिका सोनी बोलीं- मेरे पास आया था ऑफर, सिख ही...   
19938  दोस्तों से भगवान कृष्ण से मुलाकात की इच्छा जता...   
2879   UK में भारतीय दूतावास पर तिरंगे का अपमान किया ...   
4325   बताया था जमाखोरी का खतरा; इच्छा नहीं होने पर भ...   

                                                 Summary  \
9479   Rajasthan Vidhan Sabha Election 2023 BJP Strat...   
13586  अंबिका सोनी ने बताया कि उन्हें सीएम पद के लिए ...   
19938  महिला का नाम टिटिआना ख्मेलोवस्क्या है, पासपोर्...   
2879   वारिस पंजाब दे मुखी अमृतपाल सिंह का हैंडलर और ...   
4325   2000 के नोट लाने के पक्ष में नहीं थे मोदी , बत...   

                                                 Article  
9479   विधान

In [ ]:
text = val_data['Article']
goldsummary = val_data['Summary']

**Note**: Mostly data is cleaned except few numerical values such as days and count, and maybe it is best not to remove them becuase summary also contains these numbers and it can be valuable for text generation.

### **Model1 : BART**

In [ ]:
bart_checkpoint = "ai4bharat/IndicBART"

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
tokenizer_bart = AutoTokenizer.from_pretrained(bart_checkpoint,
                                               do_lower_case=False,
                                               use_fast=False,
                                               keep_accents=True)

model_bart = AutoModelForSeq2SeqLM.from_pretrained(bart_checkpoint).to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
bos_id = tokenizer_bart._convert_token_to_id_with_added_voc("<s>")
eos_id = tokenizer_bart._convert_token_to_id_with_added_voc("</s>")
pad_id = tokenizer_bart._convert_token_to_id_with_added_voc("<pad>")

In [ ]:
from tqdm import tqdm

In [ ]:
%%time

# SystemSummary = []

# for i, input_text in enumerate(text):

#     inp = tokenizer_bart(input_text, add_special_tokens=False,
#                          truncation=True, return_tensors="pt",
#                          padding='max_length', max_length=1024)['input_ids'].to(device)

#     model_output = model_bart.generate(inp, use_cache=True,
#                                         num_beams=4,
#                                         max_length=70,
#                                         min_length=30,
#                                         early_stopping=True,
#                                         pad_token_id=pad_id,
#                                         bos_token_id=bos_id,
#                                         eos_token_id=eos_id,
#                                         decoder_start_token_id=tokenizer_bart._convert_token_to_id_with_added_voc("<2en>"))

#     decoded_output = tokenizer_bart.decode(model_output[0],
#                                     skip_special_tokens=True,
#                                     clean_up_tokenization_spaces=False)

#     SystemSummary.append(decoded_output)


SystemSummary = []

# Wrap your loop with tqdm for a progress bar
for i, input_text in tqdm(enumerate(text), total=len(text), desc="Processing Texts"):
    if not input_text.strip():
        SystemSummary.append("No content to summarize")
        continue

    try:
        inp = tokenizer_bart(input_text, add_special_tokens=False,
                             truncation=True, return_tensors="pt",
                             padding='max_length', max_length=1024)['input_ids'].to(device)

        if inp.numel() == 0:
            SystemSummary.append("No tokens generated from text")
            continue

        model_output = model_bart.generate(inp, use_cache=True, num_beams=4,
                                           max_length=70, min_length=30,
                                           early_stopping=True, pad_token_id=pad_id,
                                           bos_token_id=bos_id, eos_token_id=eos_id,
                                           decoder_start_token_id=tokenizer_bart._convert_token_to_id_with_added_voc("<2en>"))

        decoded_output = tokenizer_bart.decode(model_output[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
        SystemSummary.append(decoded_output)

    except Exception as e:
        print(f"Error processing input at index {i}: {str(e)}")
        SystemSummary.append("Error in processing text")

print("Summary generation completed.")


Processing Texts: 100%|██████████| 2123/2123 [20:21<00:00,  1.74it/s]

Summary generation completed.
CPU times: user 20min 22s, sys: 2.35 s, total: 20min 25s
Wall time: 20min 21s


In [ ]:
Summaries = pd.DataFrame(list(zip(goldsummary, SystemSummary)), columns =['GoldSummary', 'IndicBARTSummary'])
Summaries

,GoldSummary,IndicBARTSummary
0,उत्तर भारत के हिस्सों में 29-31 दिसंबर से रात ...,नयी दिल्ली: उत्तर भारत के हिस्सों में 29-31 दि...
1,शनिवार की दोपहर करीब एक बजे सैकड़ों लोगों की म...,मुजफ्फरनगर: उत्तर प्रदेश के मंत्री सुरेश राणा ...
2,UP Bhadohi Bahubali Ex-MLA Vijay Mishra इस वक्...,प्रयागराज में अतीक अहमद और अशरफ की हत्या के 3 ...
3,''दिल्ली चलो'' आह्वान के तहत राष्ट्रीय राजधानी...,"चंडीगढ़। ""दिल्ली चलो"" आह्वान के तहत राष्ट्रीय ..."
4,उत्तर प्रदेश के मंत्री कपिल देव अग्रवाल के भाई...,लखनऊ: उत्तर प्रदेश के मंत्री कपिल देव अग्रवाल ...
...,...,...
2118,बिहार लोक सेवा आयोग ने 69वें कंबाइंड कांपटीटिव...,बिहार लोक सेवा आयोग ने 69वें कंबाइंड कॉम्पिटिट...
2119,TruthGPT vs ChatGPT; Elon Musk's Launch of Tru...,बिलिनेयर एलन मस्क ने आर्टिफिशियल इंटेलिजेंस (A...
2120,Nita Mukesh Ambani Cultural Centre (NMACC) - T...,हाल ही में नीता मुकेश अंबानी कल्चरल सेंटर में ...
2121,"कामरूप मेट्रोपोलिटन, बारपेटा, धुबरी और करीमगंज...",गुवाहाटी. अफगानिस्तान पर तालिबान के कब्जे का स...


In [ ]:
Summaries.to_csv(path + "datasets/" + 'results_IndicBART.csv', sep=';', header=True, index=False, columns=['GoldSummary', 'IndicBARTSummary'])

In [ ]:
rouge = Rouge()
score = rouge.get_scores(Summaries['IndicBARTSummary'], Summaries['GoldSummary'], avg=True)
BartRouge = pd.DataFrame(score).set_index([['recall','precision','f-measure']])
BartRouge

,rouge-1,rouge-2,rouge-l
recall,0.639737,0.551966,0.604298
precision,0.493221,0.393840,0.466027
f-measure,0.548568,0.451533,0.518336


### **Model2 : mT5**

In [ ]:
t5_checkpoint = "csebuetnlp/mT5_multilingual_XLSum"

tokenizer_t5 = AutoTokenizer.from_pretrained(t5_checkpoint)
model_t5 = AutoModelForSeq2SeqLM.from_pretrained(t5_checkpoint).to(device)

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:550: UserWarning:

The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.



In [ ]:
%%time

# SystemSummary1 = []

# for i, input_text in enumerate(text):

#     input_ids = tokenizer_t5(input_text,
#                              return_tensors="pt",
#                              padding="max_length",
#                              truncation=True,
#                              max_length=768)["input_ids"].to(device)

#     model_output = model_t5.generate(input_ids=input_ids,
#                                      num_beams=4,
#                                      max_length=70,
#                                      min_length=30,
#                                      no_repeat_ngram_size=2,
#                                      early_stopping=True)

#     decoded_output = tokenizer_t5.decode(model_output[0],
#                                   skip_special_tokens=True,
#                                   clean_up_tokenization_spaces=False)

#     SystemSummary1.append(decoded_output)


SystemSummary1 = []

# Use tqdm for the status bar
for i, input_text in tqdm(enumerate(text), total=len(text), desc="Generating Summaries"):
    # Check if the text is empty or contains only whitespace
    if not input_text.strip():
        SystemSummary1.append("No content to summarize")
        continue

    try:
        input_ids = tokenizer_t5(input_text,
                                 return_tensors="pt",
                                 padding="max_length",
                                 truncation=True,
                                 max_length=768)["input_ids"].to(device)

        # Check if input_ids tensor is empty
        if input_ids.nelement() == 0:  # .nelement() returns the total number of elements in input tensor
            SystemSummary1.append("No tokens generated from text")
            continue

        model_output = model_t5.generate(input_ids=input_ids,
                                         num_beams=4,
                                         max_length=70,
                                         min_length=30,
                                         no_repeat_ngram_size=2,
                                         early_stopping=True)

        decoded_output = tokenizer_t5.decode(model_output[0],
                                             skip_special_tokens=True,
                                             clean_up_tokenization_spaces=False)

        SystemSummary1.append(decoded_output)

    except Exception as e:
        print(f"Error processing input at index {i}: {str(e)}")
        SystemSummary1.append("Error in processing text")

print("Summary generation completed.")


Generating Summaries: 100%|██████████| 2123/2123 [43:19<00:00,  1.22s/it]

Summary generation completed.
CPU times: user 43min 22s, sys: 4.59 s, total: 43min 27s
Wall time: 43min 19s


In [ ]:
Summaries['T5Summary'] = SystemSummary1

In [ ]:
Summaries.head()

,GoldSummary,IndicBARTSummary,T5Summary
0,उत्तर भारत के हिस्सों में 29-31 दिसंबर से रात ...,नयी दिल्ली: उत्तर भारत के हिस्सों में 29-31 दि...,भारत में मौसम विज्ञान विभाग का कहना है कि उत्त...
1,शनिवार की दोपहर करीब एक बजे सैकड़ों लोगों की म...,मुजफ्फरनगर: उत्तर प्रदेश के मंत्री सुरेश राणा ...,उत्तर प्रदेश के कैबिनेट मंत्री सुरेश राणा के प...
2,UP Bhadohi Bahubali Ex-MLA Vijay Mishra इस वक्...,प्रयागराज में अतीक अहमद और अशरफ की हत्या के 3 ...,उत्तर प्रदेश के मुख्यमंत्री योगी आदित्यनाथ और ...
3,''दिल्ली चलो'' आह्वान के तहत राष्ट्रीय राजधानी...,"चंडीगढ़। ""दिल्ली चलो"" आह्वान के तहत राष्ट्रीय ...",हरियाणा पुलिस ने कहा है कि एक ट्रक दुर्घटना मे...
4,उत्तर प्रदेश के मंत्री कपिल देव अग्रवाल के भाई...,लखनऊ: उत्तर प्रदेश के मंत्री कपिल देव अग्रवाल ...,उत्तर प्रदेश के कौशल विकास राज्य मंत्री कपिल द...


In [ ]:
Summaries.to_csv(path + "datasets/" + 'results_Baseline.csv', sep=';', header=True, index=False, columns=['GoldSummary', 'IndicBARTSummary', 'T5Summary'])

In [ ]:
rouge = Rouge()
score = rouge.get_scores(Summaries['T5Summary'], Summaries['GoldSummary'], avg=True)
T5Rouge = pd.DataFrame(score).set_index([['recall','precision','f-measure']])
T5Rouge

,rouge-1,rouge-2,rouge-l
recall,0.298973,0.167162,0.261374
precision,0.493239,0.292873,0.428903
f-measure,0.362264,0.205847,0.316032


### **Model2 : mT5 Google**

In [ ]:
t5_checkpoint = "google/mt5-base"

tokenizer_t5 = AutoTokenizer.from_pretrained(t5_checkpoint)
model_t5 = AutoModelForSeq2SeqLM.from_pretrained(t5_checkpoint).to(device)

pytorch_model.bin:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
%%time

SystemSummary2 = []

# Use tqdm for the status bar
for i, input_text in tqdm(enumerate(text), total=len(text), desc="Generating Summaries"):
    # Check if the text is empty or contains only whitespace
    if not input_text.strip():
        SystemSummary2.append("No content to summarize")
        continue

    try:
        input_ids = tokenizer_t5(input_text,
                                 return_tensors="pt",
                                 padding="max_length",
                                 truncation=True,
                                 max_length=768)["input_ids"].to(device)

        # Check if input_ids tensor is empty
        if input_ids.nelement() == 0:  # .nelement() returns the total number of elements in input tensor
            SystemSummary2.append("No tokens generated from text")
            continue

        model_output = model_t5.generate(input_ids=input_ids,
                                         num_beams=4,
                                         max_length=70,
                                         min_length=30,
                                         no_repeat_ngram_size=2,
                                         early_stopping=True)

        decoded_output = tokenizer_t5.decode(model_output[0],
                                             skip_special_tokens=True,
                                             clean_up_tokenization_spaces=False)

        SystemSummary2.append(decoded_output)

    except Exception as e:
        print(f"Error processing input at index {i}: {str(e)}")
        SystemSummary2.append("Error in processing text")

print("Summary generation completed.")

Generating Summaries: 100%|██████████| 2123/2123 [30:36<00:00,  1.16it/s]

Summary generation completed.
CPU times: user 30min 38s, sys: 2.98 s, total: 30min 41s
Wall time: 30min 36s


In [ ]:
Summaries['T5GoogleSummary'] = SystemSummary2

In [ ]:
Summaries.to_csv(path + "datasets/" + 'results_Baseline2.csv', sep=';', header=True, index=False, columns=['GoldSummary', 'IndicBARTSummary', 'T5Summary', 'T5GoogleSummary'])

In [ ]:
rouge = Rouge()
score = rouge.get_scores(Summaries['T5GoogleSummary'], Summaries['GoldSummary'], avg=True)
T5GoogleRouge = pd.DataFrame(score).set_index([['recall','precision','f-measure']])
T5GoogleRouge

,rouge-1,rouge-2,rouge-l
recall,0.135256,0.041045,0.106331
precision,0.265503,0.089095,0.210077
f-measure,0.175285,0.054817,0.138105


In [ ]:
T5GoogleRouge = T5GoogleRouge[['rouge-1','rouge-l', 'rouge-2']]
T5Rouge = T5Rouge[['rouge-1','rouge-l', 'rouge-2']]
BartRouge = BartRouge[['rouge-1','rouge-l', 'rouge-2']]

### **Plotting Rouge Score**

In [ ]:
!pip install -U kaleido

In [ ]:
fig = px.bar(T5GoogleRouge*100, x=T5GoogleRouge.index, y=T5GoogleRouge.columns,
             barmode='group',
             text_auto='.2s',
             labels={
                     "Algo": "Algorithms",
                     "value": "Rouge Score",
                     "variable": "legend",
                     'index': "Metrics"
                 })

fig.update_layout(width=650,
                  height=400,
                  title={
                  'text': "Score",
                  'y':.96,
                  'x':0.49,
                  'xanchor': 'center',
                  'yanchor': 'top'})

fig.show()
fig.write_html(path + 'mT5Google_plot.html')
# fig.write_image(path + 'mT5Google_plot.png')

In [ ]:
fig = px.bar(T5Rouge*100, x=T5Rouge.index, y=T5Rouge.columns,
             barmode='group',
             text_auto='.2s',
             labels={
                     "Algo": "Algorithms",
                     "value": "Rouge Score",
                     "variable": "legend",
                     'index': "Metrics"
                 })

fig.update_layout(width=650,
                  height=400,
                  title={
                  'text': "Score",
                  'y':.96,
                  'x':0.49,
                  'xanchor': 'center',
                  'yanchor': 'top'})

fig.show()
fig.write_html(path + 'mT5_plot.html')
# fig.write_image(path + 'mT5_plot.png')

![newplot.png](attachment:41c6ca4b-c8d5-43ae-a9f3-4436725df739.png)

In [ ]:
fig = px.bar(BartRouge*100, x=BartRouge.index, y=BartRouge.columns,
             barmode='group',
             text_auto='.2s',
             labels={
                     "Algo": "Algorithms",
                     "value": "Rouge Score",
                     "variable": "legend",
                     'index': "Metrics"
                 })
fig.update_layout( width=650,
                  height=400,
                title={
                  'text': "Score",
                  'y':.96,
                  'x':0.49,
                  'xanchor': 'center',
                  'yanchor': 'top'})

fig.show()
fig.write_html(path + 'IndicBART_plot.html')
# fig.write_image(path + 'IndicBART_plot.png')

![newplot (1).png](attachment:fbccaf73-89cf-4f36-ad97-52099df820d2.png)

### **Sample Summaries**

In [ ]:
[i for i in Summaries['GoldSummary'][:5]]

['उत्तर भारत के हिस्सों में 29-31 दिसंबर से रात के तापमान में 3-5 डिग्री सेल्सियस तक की गिरावट दर्ज की जा सकती है। कई राज्यों में शीतलहर चल रही है। यह बात सोमवार को भारत मौसम विभाग (आईएमडी) ने कही।',
 'शनिवार की दोपहर करीब एक बजे सैकड़ों लोगों की मौजूदगी में मंत्री सुरेश राणा के पिता ठाकुर रणवीर सिंह का अंतिम संस्कार किया गया।',
 'UP Bhadohi Bahubali Ex-MLA Vijay Mishra इस वक्त विजय मिश्रा, उनका बेटा विष्णु मिश्रा, भतीजा मनीष मिश्रा जेल में बंद हैं। इन सबकी करोड़ों रुपए की संपत्ति जब्त हो चुकी है।',
 "''दिल्ली चलो'' आह्वान के तहत राष्ट्रीय राजधानी जा रहे किसानों के ट्रैक्टर को शुक्रवार को हरियाणा के भिवानी जिले में एक ट्रक ने टक्कर मार दी, जिसमें एक किसान की मौत हो गई और दो अन्य घायल हो गए।",
 'उत्तर प्रदेश के मंत्री कपिल देव अग्रवाल के भाई के खिलाफ प्रधानमंत्री नरेंद्र मोदी और मुख्यमंत्री योगी आदित्यनाथ की तस्वीरें लगाकर मोबाइल फोन का प्रचार करने के मामले में धोखाधड़ी और जालसाजी का मुकदमा दर्ज किया गया है।']

In [ ]:
[i for i in Summaries['T5Summary'][:5]]

['भारत में मौसम विज्ञान विभाग का कहना है कि उत्तर भारत के हिस्सों में 29-31 दिसंबर से रात के तापमान में 3-5 डिग्री सेल्सियस तक की गिरावट दर्ज की जा सकती है।',
 'उत्तर प्रदेश के कैबिनेट मंत्री सुरेश राणा के पिता रणबीर सिंहराणा का निधन हो गया है।',
 'उत्तर प्रदेश के मुख्यमंत्री योगी आदित्यनाथ और उनकी पत्नी मुलायम सिंह यादव ने साल 1984 में प्रयागराज में दो महिलाओं की हत्या कर दी थी।',
 'हरियाणा पुलिस ने कहा है कि एक ट्रक दुर्घटना में एक किसान की मौत हो गई और दो अन्य घायल हुए हैं।',
 'उत्तर प्रदेश के कौशल विकास राज्य मंत्री कपिल देव अग्रवाल के भाई के खिलाफ धोखाधड़ी और जालसाजी का मुकदमा दर्ज किया गया है।']

In [ ]:
[i for i in Summaries['IndicBARTSummary'][:5]]

['नयी दिल्ली: उत्तर भारत के हिस्सों में 29-31 दिसंबर से रात के तापमान में 3-5 डिग्री सेल्सियस तक की गिरावट दर्ज की जा सकती है। कई राज्यों में शीतलहर चल रही है। यह बात सोमवार को भारत मौसम विभाग (आईएमडी) ने कही। आईएमडी ने कहा कि कि शीतलहर की स्थिति में दो जनवरी',
 'मुजफ्फरनगर: उत्तर प्रदेश के मंत्री सुरेश राणा के पिता रणबीर सिंह राणा का निधन हो गया। वे 92 वर्ष के थे। वे कुछ समय से बीमार थे। सुरेश राणा ने बताया, ‘‘मेरे पिता का शुक्रवार की रात शामली जिले के थाना भवन में निधन हो गया।’’ उन्होंने बताया कि शनिवार को उनका अंतिम संस्कार कर दिया गया।',
 'प्रयागराज में अतीक अहमद और अशरफ की हत्या के 3 दिन बाद यूपी सरकार ने माफियाओं की एक लिस्ट जारी की है। इसमें 61 गैंगस्टर्स और बाहुबलियों के नाम हैं। कुछ जेल में हैं तो कुछ फरार हैं। इस लिस्ट का पहला नाम पूर्व विधायक विजय मिश्रा का है। 65 से ज्यादा केस दर्ज हैं',
 'चंडीगढ़। "दिल्ली चलो" आह्वान के तहत राष्ट्रीय राजधानी जा रहे किसानों के ट्रैक्टर को शुक्रवार को हरियाणा के भिवानी जिले में एक ट्रक ने टक्कर मार दी, जिसमें एक किसान की मौत हो गई और दो अन्य

Note: Plotly plots are not rendering after saving notebook, so I kept same graph as image file.

In [ ]:
[i for i in Summaries['T5GoogleSummary'][:5]]

['<extra_id_0> होगी। नयी दिल्ली: <extra_id_1> भारत में शीतलहर चल रही है। आईएमडी ने कहा कि, ‘‘अगले तीन दिनों में तापमान में कमी आई है।’’',
 '<extra_id_0> है। उन्होंने आगे लिखा था। <extra_id_1> शांति! उत्तर प्रदेश के मंत्री सुरेश राणा का निधन <extra_id_2>।',
 '<extra_id_0> थे। अब राजनीति में आ जाओ। <extra_id_1> के लिए तैयार हैं। कहानी शुरू होती है... तब सीएम कमलापति ने कहा था- विजय धंधा तो चलता रहेगा... <extra_id_2>े',
 '<extra_id_0> है। यह भी पढ़ें- कश्मी <extra_id_1>, नई दिल्ली, किसानों के ट्रैक्टर को टक्कर मार दी।',
 '<extra_id_0> है। CBI जांच की मांग की गई। <extra_id_1> लल्लू ने मांगी थी माफी <extra_id_2> -']